In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  2.06it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 183.47 Mb (0.0% reduction)


In [3]:
from time_utils import date_add_days

hist_max_date = df_hist.groupby(key)['purchase_date'].max()
hist_min_date = df_hist.groupby(key)['purchase_date'].min()
hist_max_date.name = 'hist_purchase_date_max'
hist_min_date.name = 'hist_purchase_date_min'
hist_max_date = hist_max_date.to_frame()
hist_min_date = hist_min_date.to_frame()

# month_maxは翌月の数字にする
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_month_max'].map(lambda x: str(x)[:7])

hist_min_date['hist_purchase_month_min'] = hist_min_date['hist_purchase_date_min'].map(lambda x: str(x)[:7])


new_max_date = df_new.groupby(key)['purchase_date'].max()
new_min_date = df_new.groupby(key)['purchase_date'].min()
new_max_date.name = 'new_purchase_date_max'
new_min_date.name = 'new_purchase_date_min'
new_max_date = new_max_date.to_frame()
new_min_date = new_min_date.to_frame()

# month_maxは翌月の数字にする
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_month_max'].map(lambda x: str(x)[:7])
new_min_date['new_purchase_month_min'] = new_min_date['new_purchase_date_min'].map(lambda x: str(x)[:7])

df_hist.set_index(key, inplace=True)
df_new.set_index(key, inplace=True)
df_hist = df_hist.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)
df_new = df_new.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)

df_hist['hist_purchase_month_max'] = pd.to_datetime(df_hist['hist_purchase_month_max'])
df_hist['hist_purchase_month_min'] = pd.to_datetime(df_hist['hist_purchase_month_min'])
df_hist['new_purchase_month_max'] = pd.to_datetime(df_hist['new_purchase_month_max'])
df_hist['new_purchase_month_min'] = pd.to_datetime(df_hist['new_purchase_month_min'])

df_new['new_purchase_month_max'] = pd.to_datetime(df_new['new_purchase_month_max'])
df_new['new_purchase_month_min'] = pd.to_datetime(df_new['new_purchase_month_min'])
df_new['hist_purchase_month_max'] = pd.to_datetime(df_new['hist_purchase_month_max'])
df_new['hist_purchase_month_min'] = pd.to_datetime(df_new['hist_purchase_month_min'])

In [4]:
df_train = df_train.join(hist_max_date)
df_test = df_test.join(hist_max_date)
df_train['hist_purchase_month_max'] = pd.to_datetime(df_train['hist_purchase_month_max'])
df_train['first_active_month'] = pd.to_datetime(df_train['first_active_month'])
df_test['hist_purchase_month_max'] = pd.to_datetime(df_test['hist_purchase_month_max'])
df_test['first_active_month'] = pd.to_datetime(df_test['first_active_month'])
df_train['hist_regist_term']= (df_train['hist_purchase_month_max'].map(lambda x: x.year) - df_train['first_active_month'].map(lambda x: x.year)) * 12 + (df_train['hist_purchase_month_max'].map(lambda x: x.month) - df_train['first_active_month'].map(lambda x: x.month))
df_test['hist_regist_term']= (df_test['hist_purchase_month_max'].map(lambda x: x.year) - df_test['first_active_month'].map(lambda x: x.year)) * 12 + (df_test['hist_purchase_month_max'].map(lambda x: x.month) - df_test['first_active_month'].map(lambda x: x.month))
train_test = pd.concat([df_train, df_test], axis=0)

train_test_hist_term = train_test.copy()
train_test_hist_term.head()

,feature_1,feature_2,feature_3,first_active_month,hist_purchase_date_max,hist_purchase_month_max,hist_regist_term,target
card_id,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06-01,2018-02-25 09:31:15,2018-03-01,9.0,-0.820312
C_ID_3d0044924f,4,1,0,2017-01-01,2018-01-31 22:31:09,2018-02-01,13.0,0.392822
C_ID_d639edf6cd,2,2,0,2016-08-01,2018-02-27 19:08:25,2018-03-01,19.0,0.687988
C_ID_186d6a6901,4,3,0,2017-09-01,2018-02-28 11:44:40,2018-03-01,6.0,0.142456
C_ID_cdbd2c0db2,1,3,0,2017-11-01,2018-02-28 20:40:41,2018-03-01,4.0,-0.159790


In [9]:
df_new_lag1 = df_new[df_new['month_lag']==1]
df_new_lag2 = df_new[df_new['month_lag']==2]

df = df_hist

auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]
print(auth1.shape)
print(auth0.shape)

cat1_0 = False
cat1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    new_cat1 = df_new[df_new.category_1==1]
    new_cat0 = df_new[df_new.category_1==0]
#     del auth1, auth0
#     gc.collect()
    auth1_cat1_lag0  = auth1_cat1[auth1_cat1['month_lag']==0]
    auth1_cat0_lag0  = auth1_cat0[auth1_cat0['month_lag']==0]
    auth1_cat1_lag1  = auth1_cat1[auth1_cat1['month_lag']==-1]
    auth1_cat0_lag1  = auth1_cat0[auth1_cat0['month_lag']==-1]
    auth1_cat1_lag2  = auth1_cat1[auth1_cat1['month_lag']==-2]
    auth1_cat0_lag2  = auth1_cat0[auth1_cat0['month_lag']==-2]
    auth1_cat1_lag02 = auth1_cat1[auth1_cat1['month_lag']>=-2]
    auth1_cat0_lag02 = auth1_cat0[auth1_cat0['month_lag']>=-2]
    auth1_cat1_lag05 = auth1_cat1[auth1_cat1['month_lag']>=-5]
    auth1_cat0_lag05 = auth1_cat0[auth1_cat0['month_lag']>=-5]
    
    new_cat1_lag1  = new_cat1[new_cat1['month_lag']==1]
    new_cat0_lag1  = new_cat0[new_cat0['month_lag']==1]
    new_cat1_lag2  = new_cat1[new_cat1['month_lag']==2]
    new_cat0_lag2  = new_cat0[new_cat0['month_lag']==2]

else:
    
    auth1_lag0 = auth1[auth1['month_lag']==0]
    auth1_lag1 = auth1[auth1['month_lag']==-1]
    auth1_lag2 = auth1[auth1['month_lag']==-2]
    auth1_lag02 = auth1[auth1['month_lag']>=-2]
    auth1_lag05 = auth1[auth1['month_lag']>=-5]
    
    auth0_lag0 = auth0[auth0['month_lag']==0]
    auth0_lag1 = auth0[auth0['month_lag']==-1]
    auth0_lag2 = auth0[auth0['month_lag']==-2]
    auth0_lag02 = auth0[auth0['month_lag']>=-2]
    auth0_lag05 = auth0[auth0['month_lag']>=-5]

In [10]:
prefix = '214_pst'
new_df_list = [new_cat1, new_cat0, new_cat1_lag1, new_cat0_lag1, new_cat1_lag2, new_cat0_lag2]
new_fname_list = ['new_cat1', 'new_cat0', 'new_cat1_lag1', 'new_cat0_lag1', 'new_cat1_lag2', 'new_cat0_lag2']

debug = False
if debug:
    train_test = train_test.head(10000)
        

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


#========================================================================
# 集計が必要なFeature
#========================================================================
for df, fname in zip(tqdm(new_df_list), new_fname_list):
    if debug:
        df = df.head(3000)
    
    df_train = utils.read_df_pkl('../input/train0*')
    df_test = utils.read_df_pkl('../input/test0*')
    df_train.set_index(key, inplace=True)
    df_test.set_index(key, inplace=True)
    train_test = pd.concat([df_train, df_test], axis=0)
    
    # new
    print('Make Feature.')
    
    df.reset_index(inplace=True)
#     df['price'] = df['purchase_amount'] / df['installments']
    
    aggs = {}
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['duration'] = df['purchase_amount']*df['month_diff']
    
    aggs['duration'] = ['sum', 'mean', 'std', 'max', 'skew']
    aggs['purchase_amount'] = ['sum']
    aggs['card_id'] = ['count']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    
    train_test = train_test.join(df_agg)
    print(train_test.shape)
    
    if not(fname.count('lag')):
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 2
    else:
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 1

    #========================================================================
    # Save Feature
    ignore_features = ['first_active_month', 'card_id', target]
    for col in train_test.columns:
        if col in ignore_features: continue
        if col.count('feature_'):continue
        if not(col.count('dura')) and not(col.count('CLV')):continue
        feature = train_test[col].astype('float32').values
        col = col.replace('.', '_')
    #     utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
        utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
    #========================================================================

100%|██████████| 3/3 [00:00<00:00, 172.26it/s]

Make Feature.
(325540, 12)


100%|██████████| 3/3 [00:00<00:00, 175.08it/s]

Make Feature.
(325540, 12)


100%|██████████| 3/3 [00:00<00:00, 164.67it/s]

...

100%|██████████| 3/3 [00:00<00:00, 174.44it/s]

Make Feature.
(325540, 12)


100%|██████████| 3/3 [00:00<00:00, 175.29it/s]

Make Feature.
(325540, 12)


100%|██████████| 6/6 [01:52<00:00, 18.75s/it]


In [11]:
debug = False
prefix = '214_pst'
hist_df_list = [
auth1_cat1 
,auth1_cat0 
,auth1_cat1_lag0 
,auth1_cat0_lag0 
,auth1_cat1_lag1 
,auth1_cat0_lag1 
,auth1_cat1_lag2 
,auth1_cat0_lag2 
,auth1_cat1_lag02
,auth1_cat0_lag02
,auth1_cat1_lag05
,auth1_cat0_lag05
]


hist_fname_list = [
'auth1_cat1'
,'auth1_cat0'
,'auth1_cat1_lag0'
,'auth1_cat0_lag0'
,'auth1_cat1_lag1'
,'auth1_cat0_lag1'
,'auth1_cat1_lag2'
,'auth1_cat0_lag2'
,'auth1_cat1_lag02'
,'auth1_cat0_lag02'
,'auth1_cat1_lag05'
,'auth1_cat0_lag05'
]

for df, fname in zip(tqdm(hist_df_list), hist_fname_list):
    if debug:
        df = df.head(3000)
    
#     df_train = utils.read_df_pkl('../input/train0*')
#     df_test = utils.read_df_pkl('../input/test0*')
#     df_train.set_index(key, inplace=True)
#     df_test.set_index(key, inplace=True)
#     train_test = pd.concat([df_train, df_test], axis=0)
    train_test = train_test_hist_term.copy()
    
#     hist
    print('Make Feature.')
    
    df.reset_index(inplace=True)

    aggs = {}
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['duration'] = df['purchase_amount']*df['month_diff']
    aggs['duration'] = ['sum', 'mean', 'std', 'max', 'skew']
    
    aggs['purchase_amount'] = ['sum']
    aggs['card_id'] = ['count']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    
    train_test = train_test.join(df_agg)
    print(train_test.shape)
    
    if not(fname.count('lag')):
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / train_test['hist_regist_term']
    elif fname[-4:-1]=='lag':
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 1
    elif fname[-2:]=='02':
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 3
    elif fname[-2:]=='05':
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 6
    elif fname[-2:]=='08':
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 9
    elif fname[-2:]=='11':
        train_test[f'{fname}_CLV'] = train_test[f'{fname}_card_id_count'] * train_test[f'{fname}_purchase_amount_sum'] / 12
    
    #========================================================================
    # Save Feature
    ignore_features = ['first_active_month', 'card_id', target]
    for col in train_test.columns:
        if col in ignore_features: continue
        if col.count('feature_'):continue
        if not(col.count('durat')) and not(col.count('CLV')):continue
        feature = train_test[col].astype('float32').values
        col = col.replace('.', '_')
#         utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
        utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
    #========================================================================    

  0%|          | 0/12 [00:00<?, ?it/s]

Make Feature.
(325540, 15)


  8%|▊         | 1/12 [00:16<03:05, 16.87s/it]

Make Feature.
(325540, 15)


 17%|█▋        | 2/12 [01:12<04:45, 28.58s/it]

...

 83%|████████▎ | 10/12 [05:00<01:00, 30.43s/it]

Make Feature.
(325540, 15)


 92%|█████████▏| 11/12 [05:18<00:26, 26.52s/it]

Make Feature.
(325540, 15)


100%|██████████| 12/12 [06:18<00:00, 36.71s/it]


In [3]:
prefix = '214_pst'
import glob
# #========================================================================
# # これは作成済のFeatureを読み込んで計算する
# #========================================================================
combi_list = [ 
    ['new_cat1', 'new_cat0']
    ,['new_cat1_lag1', 'new_cat0_lag1']
    ,['new_cat1_lag2', 'new_cat0_lag2']
    ,['new_cat1', 'new_cat1_lag1'] 
    ,['new_cat1', 'auth1_cat1'] 
    
    ,['new_cat1', 'auth1_cat1_lag0'] 
    ,['new_cat1', 'auth1_cat1_lag02'] 
    ,['new_cat1', 'auth1_cat1_lag05']  
    ,['new_cat1', 'auth1_cat1_lag08']  
    ,['new_cat1', 'auth1_cat1_lag011'] 
    
    ,['new_cat0', 'new_cat0_lag1'] 
    ,['new_cat0', 'auth1_cat0'] 
    ,['new_cat0', 'auth1_cat0_lag0'] 
    ,['new_cat0', 'auth1_cat0_lag02'] 
    ,['new_cat0', 'auth1_cat0_lag05']  
    ,['new_cat0', 'auth1_cat0_lag08']  
    ,['new_cat0', 'auth1_cat0_lag011'] 
    
    ,['auth1_cat1_lag0', 'auth1_cat0_lag0'] 
    ,['auth1_cat1_lag02', 'auth1_cat0_lag02'] 
    ,['auth1_cat1_lag05', 'auth1_cat0_lag05'] 
    ,['auth1_cat1_lag011', 'auth1_cat0_lag011'] 
    
    ,['auth1_cat1_lag0', 'auth1_cat1_lag02'] 
    ,['auth1_cat0_lag0', 'auth1_cat0_lag02'] 
    ,['auth1_cat1_lag0', 'auth1_cat1_lag05'] 
    ,['auth1_cat0_lag0', 'auth1_cat0_lag05'] 
    ,['auth1_cat1_lag0', 'auth1_cat1_lag08'] 
    ,['auth1_cat0_lag0', 'auth1_cat0_lag08'] 
    ,['auth1_cat1_lag0', 'auth1_cat1_lag011'] 
    ,['auth1_cat0_lag0', 'auth1_cat0_lag011'] 
    
    ,['auth1_cat1_lag02', 'auth1_cat1_lag05'] 
    ,['auth1_cat0_lag02', 'auth1_cat0_lag05'] 
    ,['auth1_cat1_lag02', 'auth1_cat1_lag08'] 
    ,['auth1_cat0_lag02', 'auth1_cat0_lag08'] 
    ,['auth1_cat1_lag02', 'auth1_cat1_lag011'] 
    ,['auth1_cat0_lag02', 'auth1_cat0_lag011'] 
    
    ,['auth1_cat1_lag05', 'auth1_cat1_lag08'] 
    ,['auth1_cat0_lag05', 'auth1_cat0_lag08'] 
    ,['auth1_cat1_lag05', 'auth1_cat1_lag011'] 
    ,['auth1_cat0_lag05', 'auth1_cat0_lag011'] 
]

comp_cols = [
     'CLV'
    ,'duration'
]

tmp_feature_list = glob.glob('../features/3_third_valid/214_*.gz')  + glob.glob('../features/4_winner/214_*.gz') + glob.glob('../features/1_first_valid/214_*.gz') + glob.glob('../features/2_second_valid/214_*.gz')
                   
feature_list = []
for f in tmp_feature_list:
    if f.count('pst_ratio_') or f.count('pst_diff_'):continue
    for col in comp_cols:
        if f.count(col):
            feature_list.append(f)
            
base = utils.read_df_pkl('../input/base_first*0*')
p_list = utils.parallel_load_data(path_list=feature_list)
df_feat = pd.concat(p_list, axis=1)
train_test = pd.concat([base[key], df_feat], axis=1)

for (fm1, fm2) in tqdm(combi_list):
    for col in comp_cols:
#         203_pst_auth0_lag02_monthly_avg_purchase_amount_min
        try:
            train_test[f"ratio_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] / train_test[f"{prefix}_{fm2}_{col}@"]
            train_test[f"diff_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] - train_test[f"{prefix}_{fm2}_{col}@"]
        except KeyError:
            print(fm1, fm2, col)
            continue
print(train_test.shape)
train_test.head()

100%|██████████| 39/39 [00:00<00:00, 176.95it/s]

new_cat1 new_cat0 duration
new_cat1_lag1 new_cat0_lag1 duration
new_cat1_lag2 new_cat0_lag2 duration
new_cat1 new_cat1_lag1 duration
new_cat1 auth1_cat1 duration
new_cat1 auth1_cat1_lag0 duration
new_cat1 auth1_cat1_lag02 duration
new_cat1 auth1_cat1_lag05 duration
new_cat1 auth1_cat1_lag08 CLV
new_cat1 auth1_cat1_lag08 duration
new_cat1 auth1_cat1_lag011 CLV
new_cat1 auth1_cat1_lag011 duration
new_cat0 new_cat0_lag1 duration
new_cat0 auth1_cat0 duration
new_cat0 auth1_cat0_lag0 duration
new_cat0 auth1_cat0_lag02 duration
new_cat0 auth1_cat0_lag05 duration
new_cat0 auth1_cat0_lag08 CLV
new_cat0 auth1_cat0_lag08 duration
new_cat0 auth1_cat0_lag011 CLV
new_cat0 auth1_cat0_lag011 duration
auth1_cat1_lag0 auth1_cat0_lag0 duration
auth1_cat1_lag02 auth1_cat0_lag02 duration
auth1_cat1_lag05 auth1_cat0_lag05 duration
auth1_cat1_lag011 auth1_cat0_lag011 CLV
auth1_cat1_lag011 auth1_cat0_lag011 duration
auth1_cat1_lag0 auth1_cat1_lag02 duration
auth1_cat0_lag0 auth1_cat0_lag02 duration
auth1_cat

,card_id,214_pst_auth1_cat1_lag1_CLV@,214_pst_auth1_cat1_lag2_duration_mean@,214_pst_auth1_cat0_CLV@,214_pst_auth1_cat0_lag1_duration_sum@,214_pst_auth1_cat1_lag0_duration_std@,214_pst_auth1_cat0_lag1_duration_mean@,214_pst_auth1_cat1_lag05_duration_max@,214_pst_auth1_cat1_lag2_duration_sum@,214_pst_auth1_cat0_lag02_duration_max@,214_pst_auth1_cat1_lag05_duration_sum@,214_pst_auth1_cat1_lag0_duration_max@,214_pst_auth1_cat1_lag2_CLV@,214_pst_auth1_cat0_lag05_duration_skew@,214_pst_auth1_cat1_lag02_duration_mean@,214_pst_auth1_cat1_lag0_duration_sum@,214_pst_auth1_cat0_duration_max@,214_pst_auth1_cat1_lag05_duration_mean@,214_pst_auth1_cat1_lag0_CLV@,214_pst_auth1_cat0_lag0_duration_mean@,214_pst_auth1_cat0_duration_skew@,214_pst_auth1_cat1_lag0_duration_mean@,214_pst_auth1_cat1_lag1_duration_sum@,214_pst_auth1_cat0_lag2_duration_max@,214_pst_auth1_cat0_lag1_duration_skew@,214_pst_auth1_cat1_lag05_duration_skew@,214_pst_auth1_cat0_duration_std@,214_pst_auth1_cat0_lag0_CLV@,214_pst_auth1_cat1_lag05_duration_std@,214_pst_auth1_cat1_lag02_duration_std@,214_pst_auth1_cat0_lag02_CLV@,214_pst_auth1_cat0_lag2_duration_mean@,214_pst_auth1_cat1_lag1_duration_std@,214_pst_auth1_cat0_lag0_duration_std@,214_pst_auth1_cat0_lag0_duration_skew@,214_pst_auth1_cat1_lag2_duration_skew@,214_pst_auth1_cat0_lag0_duration_sum@,214_pst_auth1_cat0_duration_mean@,214_pst_auth1_cat1_lag02_duration_max@,214_pst_auth1_cat0_lag2_duration_std@,214_pst_auth1_cat1_lag2_duration_max@,214_pst_auth1_cat1_lag02_CLV@,214_pst_auth1_cat0_lag2_duration_sum@,214_pst_auth1_cat0_lag05_duration_sum@,214_pst_auth1_cat0_lag02_duration_mean@,214_pst_auth1_cat1_lag1_duration_max@,214_pst_auth1_cat0_lag05_duration_mean@,214_pst_auth1_cat0_lag0_duration_max@,214_pst_auth1_cat0_duration_sum@,214_pst_auth1_cat1_lag0_duration_skew@,214_pst_auth1_cat0_lag02_duration_sum@,214_pst_auth1_cat0_lag02_duration_skew@,214_pst_auth1_cat0_lag05_CLV@,214_pst_auth1_cat0_lag05_duration_max@,214_pst_auth1_cat1_lag1_duration_skew@,214_pst_auth1_cat1_lag05_CLV@,214_pst_auth1_cat0_lag2_duration_skew@,214_pst_auth1_cat0_lag1_duration_max@,214_pst_auth1_cat0_lag1_duration_std@,214_pst_auth1_cat0_lag2_CLV@,214_pst_auth1_cat1_lag2_duration_std@,214_pst_auth1_cat1_lag02_duration_skew@,214_pst_auth1_cat1_lag1_duration_mean@,214_pst_auth1_cat0_lag1_CLV@,214_pst_auth1_cat1_lag02_duration_sum@,214_pst_auth1_cat0_lag02_duration_std@,214_pst_auth1_cat0_lag05_duration_std@,214_pst_new_cat0_lag1_CLV@,214_pst_auth1_cat1_duration_mean@,214_pst_auth1_cat1_duration_std@,214_pst_new_cat0_CLV@,214_pst_auth1_cat1_duration_skew@,214_pst_auth1_cat1_duration_max@,214_pst_new_cat1_lag1_CLV@,214_pst_new_cat0_duration_mean@,214_pst_new_cat1_duration_sum@,214_pst_new_cat1_CLV@,214_pst_new_cat0_lag1_duration_sum@,214_pst_new_cat0_lag1_duration_max@,214_pst_new_cat1_lag1_duration_skew@,214_pst_new_cat0_duration_std@,214_pst_new_cat1_lag1_duration_sum@,214_pst_new_cat1_duration_std@,214_pst_new_cat1_lag2_duration_mean@,214_pst_new_cat1_lag2_duration_skew@,214_pst_new_cat1_lag2_duration_sum@,214_pst_new_cat1_lag1_duration_mean@,214_pst_auth1_cat1_CLV@,214_pst_new_cat0_duration_skew@,214_pst_new_cat1_lag2_duration_std@,214_pst_new_cat1_duration_skew@,214_pst_new_cat1_lag2_CLV@,214_pst_new_cat0_lag1_duration_std@,214_pst_new_cat0_duration_max@,214_pst_new_cat0_lag2_duration_max@,214_pst_new_cat0_lag1_duration_skew@,214_pst_new_cat0_lag2_CLV@,214_pst_auth1_cat1_duration_sum@,214_pst_new_cat0_lag2_duration_sum@,214_pst_new_cat0_lag2_duration_std@,214_pst_new_cat1_duration_mean@,214_pst_new_cat1_lag1_duration_std@,214_pst_new_cat0_lag1_duration_mean@,214_pst_new_cat0_duration_sum@,214_pst_new_cat1_lag2_duration_max@,214_pst_new_cat0_lag2_duration_skew@,214_pst_new_cat1_lag1_duration_max@,214_pst_new_cat0_lag2_duration_mean@,214_pst_new_cat1_duration_max@,ratio_new_cat1_new_cat0_CLV,diff_new_cat1_new_cat0_CLV,ratio_new_cat1_lag1_new_cat0_lag1_CLV,diff_new_cat1_lag1_new_cat0_lag1_CLV,ratio_new_cat1_lag2_new_cat0_lag2_CLV,diff_new_cat1

In [5]:
### Save Feature

for col in train_test.columns:
#     if col.count('213_'):continue
    if not(col.count('ratio')) and not(col.count('diff_')):continue
    feature = train_test[col].astype('float32').values
    if np.std(feature)==0:
        continue
    
#     utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
    utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)